<a href="https://colab.research.google.com/github/Akshay-hub-007/Deforest-Detection/blob/main/deforest-dectection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

df = pd.read_csv("/content/6344ec94bb4449051550.csv")


df.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,30.1138,80.0756,300.0,1.2,1.1,2022-01-01,511,Terra,MODIS,7,6.03,288.4,7.1,D,0
1,23.7726,86.2078,306.1,1.6,1.2,2022-01-01,512,Terra,MODIS,62,6.03,293.5,10.4,D,2
2,22.2080,84.8627,304.8,1.4,1.2,2022-01-01,512,Terra,MODIS,42,6.03,293.3,5.8,D,2
3,23.7621,86.3946,306.9,1.6,1.2,2022-01-01,512,Terra,MODIS,38,6.03,295.2,9.3,D,2
4,23.6787,86.0891,303.6,1.5,1.2,2022-01-01,512,Terra,MODIS,52,6.03,293.1,7.2,D,2


In [38]:
data = df.drop(['scan', 'track','instrument','satellite','version'], axis=1)

In [39]:
data

,latitude,longitude,brightness,acq_date,acq_time,confidence,bright_t31,frp,daynight,type
0,30.1138,80.0756,300.0,2022-01-01,511,7,288.4,7.1,D,0
1,23.7726,86.2078,306.1,2022-01-01,512,62,293.5,10.4,D,2
2,22.2080,84.8627,304.8,2022-01-01,512,42,293.3,5.8,D,2
3,23.7621,86.3946,306.9,2022-01-01,512,38,295.2,9.3,D,2
4,23.6787,86.0891,303.6,2022-01-01,512,52,293.1,7.2,D,2
...,...,...,...,...,...,...,...,...,...,...
81520,29.9033,74.9521,303.3,2022-12-31,1726,55,281.3,37.2,N,0
81521,29.9086,74.9460,323.9,2022-12-31,1726,100,282.0,106.5,N,0
81522,29.9170,74.9576,323.5,2022-12-31,2127,100,274.6,66.9,N,0
81523,21.1048,72.6415,302.9,2022-12-31,2130,52,289.9,18.5,N,2


In [7]:
data.dropna(inplace=True)

In [17]:
data['daynight']=data['daynight'].map({'D':0,'N':1})

In [20]:

data['daynight'].value_counts()

,count
daynight,
0,71060
1,10465


In [36]:
data=data.dropna()

In [34]:
data

,latitude,longitude,brightness,acq_date,acq_time,confidence,bright_t31,frp,daynight,type
0,30.1138,80.0756,300.0,2022-01-01,511,7,288.4,7.1,0,0
1,23.7726,86.2078,306.1,2022-01-01,512,62,293.5,10.4,0,1
2,22.2080,84.8627,304.8,2022-01-01,512,42,293.3,5.8,0,1
3,23.7621,86.3946,306.9,2022-01-01,512,38,295.2,9.3,0,1
4,23.6787,86.0891,303.6,2022-01-01,512,52,293.1,7.2,0,1
...,...,...,...,...,...,...,...,...,...,...
81520,29.9033,74.9521,303.3,2022-12-31,1726,55,281.3,37.2,1,0
81521,29.9086,74.9460,323.9,2022-12-31,1726,100,282.0,106.5,1,0
81522,29.9170,74.9576,323.5,2022-12-31,2127,100,274.6,66.9,1,0
81523,21.1048,72.6415,302.9,2022-12-31,2130,52,289.9,18.5,1,1


In [40]:
data['type'].value_counts()

,count
type,
0,77072
2,4437
3,16


In [41]:
data['type']=data['type'].replace({2:1,3:1})

In [25]:
features = ['brightness', 'bright_t31', 'frp', 'confidence', 'daynight', 'latitude', 'longitude']
x=data[features]
y=data['type']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [ ]:
rf=RandomForestClassifier(n_estimators=100, random_state=42)

In [28]:
for n in [10, 50, 100, 200, 300, 500]:
    model = RandomForestClassifier(n_estimators=n, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{n} trees → Accuracy: {accuracy_score(y_test, y_pred):.4f}")

10 trees → Accuracy: 0.9839
50 trees → Accuracy: 0.9846
100 trees → Accuracy: 0.9845
200 trees → Accuracy: 0.9844
300 trees → Accuracy: 0.9845
500 trees → Accuracy: 0.9844


In [31]:
model=RandomForestClassifier(n_estimators=100,class_weight='balanced', random_state=42)
model.fit(X_train,y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)